# Cognira Exam 
<h6>Predict Sale's Promotions<h6>

In [2]:

import numpy as np

#Fetching the dataset
import pandas as pd


#import transaction data and the promos data
data_transact = pd.read_csv('transactions.csv',delimiter=',', encoding="utf-8-sig")
data_promo = pd.read_csv('promos.csv',delimiter=',', encoding="utf-8-sig")

#view top 10 rows 
data_transact.head(10)


FileNotFoundError: File b'transactions.csv' does not exist

In [ ]:
#show 10 row for promotions 
data_promo.head(10)

In [ ]:
train, target = pd.DataFrame(data_transact ), pd.DataFrame(data_promo)



# Exploring data

In [ ]:
#some expoloring tasks
print("Promotions \n ",data_promo.shape,"\n")
print(data_promo.info(),"\n")
print("NAN attribute\n",data_promo.isnull().sum(),"\n")

print("Transactions \n ",data_transact.shape,"\n")
print(data_transact.info(),"\n")
print("NAN attribute\n",data_transact.isnull().sum(),"\n")




**NaN sales = 385266 = 96,3% of the data transactions  
This is an important percent.  
No NaN attributes for promotions Data.**


# Imputing missing sales data with 0 value

In [ ]:
#Impute the values using scikit-learn SimpleImpute Class
from sklearn.impute import SimpleImputer
imp_zeros = SimpleImputer( missing_values=np.nan , strategy='constant', fill_value=0 ,copy=True)#fill missing values with 0 constant
imp_zeros.fit(train)
imputed_train_df = imp_zeros.transform(train)

In [ ]:
imputed_train_df.shape


**The result is a plain NumPy array containing the transformed features. If we want to
put it back into a Pandas DataFrame, we apply the next instruction**

In [ ]:
imputed_tr = pd.DataFrame(imputed_train_df, columns=train.columns)
imputed_tr.head(12)

# Negative sales are interpreted as returns and should be subtracted from any aggregations of sales.

In [ ]:
print((imputed_tr['sales']<0).sum()) # how much negative sales
imputed_tr= imputed_tr.drop(imputed_tr[imputed_tr.sales<0].index) # drop them from sales (interepted as returns)

# Units can be estimated as floor(sales / price)

In [ ]:
imputed_tr['Units'] = (imputed_tr['sales']/imputed_tr['price']).astype(int)
imputed_tr.head(15)

# Aggregate the data to weekly level, so that there is one row per product-week combination



In [ ]:
 #Converting date to pandas datetime format
imputed_tr['history_date'] = pd.to_datetime(imputed_tr['history_date'])

imputed_tr['Week_Number'] = imputed_tr['history_date'].dt.week# Getting week number

# Getting year. Weeknum is common across years to we need to create unique index by using year and weeknum
imputed_tr['Year'] = imputed_tr['history_date'].dt.year

#Creating a combination of WeekOfYear and Year
imputed_tr['Week_Id'] = (imputed_tr['Year'].astype(int)-2014)*53+imputed_tr['Week_Number'].astype(int)

# Grouping based on Week_Id
df2 = imputed_tr[['sales','Week_Id']].groupby([ 'Week_Id']).sum()
df2.head(10)#visualizing 

In [3]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20,5))
plt.plot(df2)
for i in range(4):
    plt.axvline(x=(i+1)*53,color='black',linestyle='--',linewidth=0.5)
plt.show()

NameError: name 'df2' is not defined

# Create any additional features you feel would help in modeling

In [ ]:
import seaborn as sns
#plot some heatmap to find correlation among the features
corrmat = train.corr();
f , ax = plt.subplots(figsize=(5,4))
sns.heatmap(corrmat, square=True)

Each square in an heatmap shows how much any two feature correlates (is there some kind of relationship between them). 
Does the increase in one feature result in the increase of the other-->(Positive correlation) or does the increase in one result in the decrease of the other -->(Negative correlation).

The vertical range running from 0.0 to 1.0 shows you the relative size of the correlation between any two features, where hotter values are higher and means more correlation.
The diagonal almost-whitish portion of the matrix shows that all features are highly correlated to the themselves.  

Tracing the first label horizontally to the left leads to Week_number and tracing it vertically downwards, leads to Week_Id. This means the square box (3) is showing us how correlated price and Category_Id are. We can see that they’re highly correlated as the color becomes hot pink.
Tracing the second interesting label shows the correlation between Price and  Sales. We clearly see that the region is hot, meaning there’s a high correlation.
 

# we start adding significant features

Price_category: This is the price of each category 
Verdict: Important
This is a concatenation of the price and category identifier.
Reason: Some type of products categorues will generally sell more than others due to the price


Price_sales: 
This is a concatenation of the Price and sales.
Reason: Some type of product will generally sell more than others.
Verdict: Important
Reason: This is certainly an important feature because the price of products will definitely affect out the sales

In [ ]:
imputed_tr['Price_Category'] = imputed_tr[['price','category_id']].astype(str).apply('|'.join,1)
imputed_tr.head(10)

# Make sure to perform any other transformations for missing values, outliers, improperly scaled information, or anything else that would represent a best practice for data scientists.

In [ ]:
imputed_tr.isnull().sum() # make sure that we haven't any NaN attribute

# Model Creation

In [ ]:
#aggregation of two dataset : could help in modeling
#all sales with or not promotion
imputed_tr['item_id']=imputed_tr['item_id'].astype(int)

result=pd.merge(imputed_tr,data_promo,on='item_id',how='left') # left join task

# we should impute NaN attribute for sales with no promotion

result['promo_start_dt'] = pd.to_datetime(result['promo_start_dt'])
result['promo_end_dt'] = pd.to_datetime(result['promo_end_dt'])

result['promo_type'] = np.where(result['promo_start_dt']>result['history_date'], 'NO_PROMO',result['promo_type'])
result['promo_type'] = np.where(result['promo_end_dt']<result['history_date'], 'NO_PROMO',result['promo_type'])
result['promo_type'] = np.where(result['promo_type'].isnull(), 'NO_PROMO',result['promo_type'])

result['promo_type'].value_counts()

#Mapping for promo_type

d = {'NO_PROMO': 0, 'PROMO_A': 1, 'PROMO_B':2, 'PROMO_C':3}
result['promo_type'] = result['promo_type'].map(d)

result.head()


In [ ]:
p=(result['promo_type']==0).sum() # how much sales without promotion
(p/(result.shape[0]))*100
#only 0.5 percent of sales on promotion

following part:
for time_series we want to determine values of intersection(intersection of products and prices) during a time dimension that is identified either
by a shift in sales (from low-medium to high ) or by a shift in prices( a change in prices could indicate a potential promotion)